In [1]:
import os
import argparse
import pickle
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

import sys
sys.path.append('/home/sandeep_cse/Topic-Based-Image-Captioning')

from dataset.utils import load_image
from models.utils import load_pre_trained_image_model
from evaluation.predictions import generate_predictions

In [2]:
def load_data(data_dir):
    """ Load word index dictionary """
    word_idx_cache_path = os.path.join(
        data_dir, 'word_idx.pkl'
    )
    idx_word_cache_path = os.path.join(
        data_dir, 'idx_word.pkl'
    )

    word_idx_path_exists = os.path.exists(word_idx_cache_path)
    idx_word_path_exists = os.path.exists(idx_word_cache_path)
    if idx_word_path_exists and word_idx_path_exists:
        with open(word_idx_cache_path, mode='rb') as file:
            word_idx = pickle.load(file)
        with open(idx_word_cache_path, mode='rb') as file:
            idx_word = pickle.load(file)
        print("Dictionary loaded.")
    else:
        sys.exit('File containing the dictionary does not exist.')

    return word_idx, idx_word

In [3]:
# Load dictionaries
word_idx, idx_word = load_data(
    '/home/sandeep_cse/Topic-Based-Image-Captioning/dataset/processed_data'
)

Dictionary loaded.


In [6]:
# Load caption model
caption_model = load_model(
    '/home/sandeep_cse/Topic-Based-Image-Captioning/weights/caption_model.hdf5'
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [4]:
def get_model_input(topic_model_path, img_path):
    # Load pre-trained image models
    topic_model, feature_model = load_pre_trained_image_model(topic_model_path)

    # Load image batch
    image_batch = process_image(
        img_path, K.int_shape(feature_model.input)[1:3]
    )

    # Create input data
    feature_values = feature_model.predict(image_batch)
    topic_values = topic_model.predict(feature_values)

    return topic_values[0], feature_values[0]

In [5]:
def process_image(img_path, img_size):
    # Convert image to vector
    img = load_image(img_path, size=img_size, grayscale=False)

    # Create image batch
    shape = (1,) + img_size + (3,)
    image_batch = np.zeros(shape=shape, dtype=np.float32)
    image_batch[0] = img

    return image_batch

In [8]:
topic_values, feature_values =  get_model_input(
    '/home/sandeep_cse/Topic-Based-Image-Captioning/weights/topic_model.hdf5',
    '/home/sandeep_cse/sat_imgs/11.jpg'
)

Loading pre-trained image models...
Instructions for updating:
Use tf.cast instead.
Done.



In [ ]:
image_batch = process_image('/home/sandeep_cse/sat_imgs/11.jpg', (299, 299))

In [ ]:
# Create input data
feature_values = feature_model.predict(image_batch)
topic_values = topic_model.predict(feature_values)

In [ ]:
feature_values

In [ ]:
topic_values

In [10]:
predicted_caption = generate_predictions(
    topic_values,
    feature_values,
    caption_model,
    word_idx,
    idx_word,
    16
)

In [11]:
predicted_caption

'a woman standing in front of a fruit stand filled with fruit'